<a href="https://colab.research.google.com/github/AbdelwahedSouiid/Transformers/blob/main/AdvDeep_Transformers_Helpfulness2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets


In [3]:
pip install kaggle


In [4]:

from google.colab import files
files.upload()  # Choose your kaggle.json file to upload

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"aymenmsalmi","key":"ffe2251aa4fd313d1654afdec8753d14"}'}

In [6]:
!kaggle datasets download -d arhamrumi/amazon-product-reviews


Dataset URL: https://www.kaggle.com/datasets/arhamrumi/amazon-product-reviews
License(s): CC0-1.0
amazon-product-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!unzip amazon-product-reviews.zip

Archive:  amazon-product-reviews.zip
replace Reviews.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
from datasets import Dataset

In [10]:
# Reading the data from our CSV file to store it in a DataFrame
df = pd.read_csv('/content/Reviews.csv')  # Check the extracted files for the correct filename
df.head()  # Display the first few rows
print(df.shape)

(568454, 10)


In [11]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [12]:
# prompt: just select 500 row of the data set chose the most of them with HelpfulnessDenominator	not equal to 0

# Assuming df is already loaded as in the previous code

# Select 500 rows where HelpfulnessDenominator is not 0
df_subset = df[df['HelpfulnessDenominator'] != 0].head(500)

# If you need to ensure exactly 500 rows and some rows with HelpfulnessDenominator == 0 are acceptable,
# you can use the following instead:

# df_subset = df.sample(n=500) # random sampling
# df_subset = df.iloc[:500] # first 500 rows

print(df_subset.shape)
df_subset.head()

(500, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
10,11,B0001PB9FE,A3HDKO7OW0QNK4,Canadian Fan,1,1,5,1107820800,The Best Hot Sauce in the World,I don't know if it's the cactus or the tequila...


In [13]:
df=df_subset

In [14]:
print(df.columns)

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')


In [15]:
# Step 1: Data Preprocessing
# We'll classify reviews as helpful (1) if the helpfulness nominator / denominator ratio is > 0.5, otherwise not helpful (0)
df['helpful_label'] = df.apply(
    lambda row: 1 if (row['HelpfulnessDenominator'] > 0 and (row['HelpfulnessNumerator'] / row['HelpfulnessDenominator']) > 0.5) else 0,
    axis=1
)

In [16]:
# Step 2: Split the dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Text'].tolist(), df['helpful_label'].tolist(), test_size=0.2, random_state=42)

In [17]:
# Print the types and a few examples of train_texts and test_texts
print(type(train_texts), train_texts[:5])  # Show the type and first 5 examples
print(type(test_texts), test_texts[:5])     # Show the type and first 5 examples


<class 'list'> ['The first time I tried these, I thought, "WOW!" but I quickly found they were just too darn spicy and tart... Hard to enjoy the potato-chippiness of them when your mouth is under seige.', 'This is food especially designed for Cocker spaniels who have a range of health issues...I have been using this product with three of my dogs and all of them are very happy with it. It also is packaged very well and as a result stays fresh for a long period of time. Great product!', "What else do you need to know? Oatmeal, instant (make it with a half cup of low-fat milk and add raisins;nuke for 90 seconds). More expensive than Kroger store brand oatmeal and maybe a little tastier or better texture or something. It's still just oatmeal. Mmm, convenient!", 'These are seriously delicious, especially if you like plain potato chips. My only concern is they\'re very oily. I understand they\'re "kettle chips" and they\'re "fried" (small batches and all) but when you open the bag (That\'s a

In [18]:
train_texts = df['Text'].tolist()  # Ensure this is the right column name
test_texts = df['Text'].tolist()    # Similarly for the test texts


In [19]:
train_texts = [text for text in train_texts if isinstance(text, str) and text.strip()]
test_texts = [text for text in test_texts if isinstance(text, str) and text.strip()]


In [20]:
# Step 3: Tokenization using BERT's tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Tokenize the training and test data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [21]:
# Step 4: Convert data into PyTorch Dataset format
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ReviewsDataset(train_encodings, train_labels)
test_dataset = ReviewsDataset(test_encodings, test_labels)


In [22]:
# Step 5: Load the BERT model for binary classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Step 6: Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    evaluation_strategy="epoch",     # evaluate at each epoch
    report_to='none',                # Disable W&B logging
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
# Step 7: Define a data collator to pad the sequences
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [25]:
# Step 8: Set up the Trainer class
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [26]:
# Step 9: Train the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.711180
2,No log,0.562433
3,No log,0.558176


TrainOutput(global_step=75, training_loss=0.6766713460286459, metrics={'train_runtime': 1942.4766, 'train_samples_per_second': 0.618, 'train_steps_per_second': 0.039, 'total_flos': 315733266432000.0, 'train_loss': 0.6766713460286459, 'epoch': 3.0})

In [27]:
# Step 10: Evaluate the model on the test set
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)
accuracy = accuracy_score(test_labels, preds)
f1 = f1_score(test_labels, preds)

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")


Accuracy: 0.76
F1 Score: 0.8636363636363636


In [29]:
# prompt: save the model

trainer.save_model("/content/drive/MyDrive/model/saved_model")

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
# prompt: load the model and test it

from google.colab import drive
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer

# Load the saved model and tokenizer
drive.mount('/content/drive')
model_path = "/content/drive/MyDrive/model/saved_model"  # Update with your actual path

tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)

# Example usage:
def predict_helpfulness(review_text):
    inputs = tokenizer(review_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()
    return predicted_class  # 0 or 1


# Example review texts
review_texts = [
    "This product is amazing! It exceeded my expectations.",
    "I'm very disappointed with this purchase. It's not what I expected.",
    "It was okay, not great, not bad.", # Neutral example
    "Total waste of money! Do not buy." # Negative example
]

for review in review_texts:
    prediction = predict_helpfulness(review)
    print(f"Review: {review}")
    print(f"Predicted Helpfulness Label: {prediction}")  # 0 for not helpful, 1 for helpful
    print("---")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Review: This product is amazing! It exceeded my expectations.
Predicted Helpfulness Label: 1
---
Review: I'm very disappointed with this purchase. It's not what I expected.
Predicted Helpfulness Label: 1
---
Review: It was okay, not great, not bad.
Predicted Helpfulness Label: 1
---
Review: Total waste of money! Do not buy.
Predicted Helpfulness Label: 1
---


In [31]:
# prompt: lookinf for not helpful and try see waht the mode predeict

# Example review texts - focusing on potentially unhelpful reviews
review_texts = [
    "This product is terrible.  I would not recommend it to anyone.",
    "Complete waste of money.  Do not buy this.",
    "The instructions were unclear and the product broke easily.",
    "I expected more from this.  Very disappointing.",
    "It arrived damaged and customer service was unhelpful.",
    "The quality is subpar, I wouldn't buy again.",
    "Not as advertised. False claims.", #Added more negative examples
    "Didn't work at all.  Useless.",
    "Cheaply made and broke after one use.", #Added more negative examples

    "This is okay, I guess.", # Neutral, borderline case
    "It was alright.", # Neutral, borderline case
    "Decent product.  Nothing special.", # Neutral, borderline case
]

for review in review_texts:
    prediction = predict_helpfulness(review)
    print(f"Review: {review}")
    print(f"Predicted Helpfulness Label: {prediction}")  # 0 for not helpful, 1 for helpful
    print("---")

Review: This product is terrible.  I would not recommend it to anyone.
Predicted Helpfulness Label: 1
---
Review: Complete waste of money.  Do not buy this.
Predicted Helpfulness Label: 1
---
Review: The instructions were unclear and the product broke easily.
Predicted Helpfulness Label: 1
---
Review: I expected more from this.  Very disappointing.
Predicted Helpfulness Label: 1
---
Review: It arrived damaged and customer service was unhelpful.
Predicted Helpfulness Label: 1
---
Review: The quality is subpar, I wouldn't buy again.
Predicted Helpfulness Label: 1
---
Review: Not as advertised. False claims.
Predicted Helpfulness Label: 0
---
Review: Didn't work at all.  Useless.
Predicted Helpfulness Label: 0
---
Review: Cheaply made and broke after one use.
Predicted Helpfulness Label: 0
---
Review: This is okay, I guess.
Predicted Helpfulness Label: 1
---
Review: It was alright.
Predicted Helpfulness Label: 0
---
Review: Decent product.  Nothing special.
Predicted Helpfulness Label: 1
